In [1]:
import google.generativeai as genai
import os
from dotenv import load_dotenv
import utils
from utils import (chat_with_model,moderation_check,load_products,get_products_info,find_user_product)

In [2]:
load_dotenv()
api_key=os.getenv("GEMINI__API_KEY")
genai.configure(api_key=api_key)

In [3]:
chat_history=[]

In [8]:
# Main user message processing:
def process_user_message(user_text):
    #1. Moderation check:
    #result=moderation_check(user_text)
    #if result=='Y':
        #return 'Your message contains inappropriate content. I cannot continue.'

    #2. Load products:
    products=load_products()
    found_list=find_user_product(user_text,products)

    #3. Get product details:
    product_info=get_products_info(found_list)

    #4. Create chatbot answer:
    System_instruction="""
    You are a helpful store assistant.
    Use the information provided to answer the customer's query."""

    messages=[
        {'role':'user','parts':[{'text':
                                 System_instruction+
                                 user_text+
                                 product_info}]},
    ]

    bot_response=chat_with_model(messages)

    #5. Quality evaluation:
    Evaluation_prompt = f"""
    You are an assistant that evaluates whether customer service agent responses sufficiently, answer customer questions, and also validates that 
    all the facts the assistant cites from the product information are correct.
    
    Customer message: {user_text}
    Product information: {product_info}
    Agent response: {bot_response}
    
    Does the response use the retrieved information correctly?
    Does the response sufficiently answer the question?
    
    Respond with a Y or N character, with no punctuation:
    Y - if the output sufficiently answers the question AND the response correctly uses product information
    N - otherwise
    
    Output a single letter only.
    Output Y or N
    """
    
    Eval_message = [
        {'role':'user','parts':[{'text':Evaluation_prompt}]},
    ]
    
    Evaluation_response = chat_with_model(Eval_message, max_tokens=1).strip().upper()
    
    #if Evaluation_response=='Y':
    return bot_response
    #else:
        #return "I'm not fully sure about the details. Let me connect you with a representative."

In [10]:
#Test run:
print("Chatbot is running. Type 'exit' to stop.\n")

while True:
    user_text = input("You: ")

    if user_text.lower() in ["exit", "quit"]:
        print("Bot: Goodbye!")
        break

    final_answer = process_user_message(user_text)

    print("Bot:", final_answer)
    print()

Chatbot is running. Type 'exit' to stop.



You:  Tell me about the TechPro Ultrabook Laptop


Bot: Certainly! Here's some information about the TechPro Ultrabook Laptop:

The **TechPro Ultrabook** is a sleek and lightweight ultrabook designed for everyday use.

*   **Category:** Computers and Laptops
*   **Brand:** TechPro
*   **Model Number:** TP-UB100
*   **Price:** $799.99
*   **Key Features:**
    *   13.3-inch display
    *   8GB RAM
    *   256GB SSD
    *   Intel Core i5 processor
*   **Warranty:** 1 year
*   **Rating:** 4.5 out of 5 stars

It's a great option if you're looking for a portable and efficient laptop!



You:  stop


Bot: Hello! I'm ready to help you as your store assistant.

Please provide me with the customer's query and the information I need to answer it, and I'll do my best to assist you!



You:  exit


Bot: Goodbye!
